In [6]:
import pandas as pd
import numpy as np
import json

# Load cleaned dataset
df = pd.read_csv("Merged_Top6_pH_Avg_Cleaned.csv")
df['Date'] = pd.to_datetime(df['Date'])

# Parameter columns (all numeric)
cols = ['Chloride as Cl', 'Calcium (Total)', 'Total Magnesium',
        'Sodium as Na', 'Potassium as K', 'Salinity as EC@25 (lab)', 'pH']

# ========== WQI Formula ==========
standards = {
    'Chloride as Cl': 250,
    'Calcium (Total)': 200,
    'Total Magnesium': 150,
    'Sodium as Na': 200,
    'Potassium as K': 12,
    'Salinity as EC@25 (lab)': 500,
    'pH': (6.5, 8.5)
}
weights = {
    'Chloride as Cl': 0.10,
    'Calcium (Total)': 0.10,
    'Total Magnesium': 0.10,
    'Sodium as Na': 0.20,
    'Potassium as K': 0.05,
    'Salinity as EC@25 (lab)': 0.25,
    'pH': 0.20
}

def calculate_score(value, std, indicator):
    if pd.isna(value):
        return 0
    if indicator == 'pH':
        low, high = std
        return 100 if low <= value <= high else 0
    else:
        return max(0, 100 - (value / std * 100))

def calculate_wqi(row):
    scores = []
    for ind in weights.keys():
        score = calculate_score(row[ind], standards[ind], ind)
        scores.append(score * weights[ind])
    return sum(scores) / sum(weights.values())

# ========== Forecast helper for 1-month ahead ==========
def forecast_one_month(series):
    series = series.dropna().sort_index()
    n = len(series)
    if n == 0:
        return np.nan, {"error": "no_data"}
    elif n == 1:
        return series.iloc[-1], {"method": "repeat_last"}
    else:
        # linear regression
        x = np.arange(n)
        coef = np.polyfit(x, series.values, 1)  # slope + intercept
        next_x = n
        y_pred = np.polyval(coef, next_x)
        return y_pred, {"method": "linear_regression", "coef": coef.tolist()}

# ========== Train models per site ==========
future_results = []
site_models = {}

# today's reference
today = pd.to_datetime("today").normalize()
next_date = today + pd.offsets.MonthBegin(1)

for site, group in df.groupby('Site ID'):
    group = group.sort_values('Date')

    future_data = {}
    site_models[site] = {}

    for col in cols:
        series = pd.Series(group[col].values, index=group['Date'])
        pred, model_info = forecast_one_month(series)
        site_models[site][col] = model_info
        future_data[col] = pred

    # Combine into DataFrame
    X_future = pd.DataFrame([future_data])
    X_future['Date'] = next_date
    X_future['WQI_star'] = X_future.apply(calculate_wqi, axis=1)
    X_future['Site ID'] = site
    future_results.append(X_future)

# Merge all sites
future_df = pd.concat(future_results, ignore_index=True)

# Risk Level
def risk_level(score):
    if score >= 70:
        return "Safe"
    elif score >= 50:
        return "Moderate"
    else:
        return "Unsafe"

future_df['Risk_Level'] = future_df['WQI_star'].apply(risk_level)

# Save forecast results and model parameters
future_df.to_csv("Future_WQI_Prediction_1Month.csv", index=False)
with open("site_model_params_1Month.json", "w") as f:
    json.dump(site_models, f, indent=4)

print("✅ 1-Month Prediction (from today) completed. Saved to Future_WQI_Prediction_1Month.csv")
print("✅ Model parameters saved to site_model_params_1Month.json")
print(future_df.head())


✅ 1-Month Prediction (from today) completed. Saved to Future_WQI_Prediction_1Month.csv
✅ Model parameters saved to site_model_params_1Month.json
   Chloride as Cl  Calcium (Total)  Total Magnesium  Sodium as Na  \
0      127.163636        58.272727        14.927273     95.400000   
1      140.681818        69.621212        12.348485    101.075758   
2      300.000000        -2.000000        68.000000     90.000000   
3       50.300000        61.100000        18.000000     36.400000   
4      110.100000        92.200000        24.500000     50.000000   

   Potassium as K  Salinity as EC@25 (lab)        pH       Date   WQI_star  \
0        4.052727               841.381818  7.438364 2025-10-01  54.776030   
1        0.713636               923.227273  7.491061 2025-10-01  54.663510   
2       27.000000              1144.000000  3.040000 2025-10-01  26.566667   
3        1.170000               578.600000  8.338000 2025-10-01  64.605500   
4        3.950000               719.000000  8.1200